<a href="https://colab.research.google.com/github/starmiami/sub-theme-code/blob/main/subtheme-code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# データ加工・処理・分析ライブラリ
import numpy as np
import numpy.random as random
import scipy as sp
from pandas import Series, DataFrame
import pandas as pd

# 可視化ライブラリ
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

# 機械学習ライブラリ
import sklearn

# 小数第3位まで表示
%precision 3

'%.3f'

In [ ]:
# colabにアップロードしたzipファイルを解凍
!unzip '/content/sample_data/minorR_dataset1_南.zip'

Archive:  /content/sample_data/minorR_dataset1_南.zip
   creating: minorR_dataset1/
  inflating: minorR_dataset1/02_itimoku_test.txt  
  inflating: minorR_dataset1/09_samuke_train.txt  
  inflating: minorR_dataset1/05_zyozu_test.txt  
  inflating: minorR_dataset1/02_itimoku_train.txt  
  inflating: minorR_dataset1/08_sinzyu_train.txt  
  inflating: minorR_dataset1/06_ninki_train.txt  
  inflating: minorR_dataset1/requirements.txt  
  inflating: minorR_dataset1/04_itizu_train.txt  
  inflating: minorR_dataset1/04_itizu_test.txt  
  inflating: minorR_dataset1/accuracy1.xlsx  
  inflating: minorR_dataset1/01_itimi_test.txt  
  inflating: minorR_dataset1/Instruction.txt  
  inflating: minorR_dataset1/sample_sbert.py  
  inflating: minorR_dataset1/07_sizyo_test.txt  
  inflating: minorR_dataset1/10_mokka_test.txt  
  inflating: minorR_dataset1/03_ikken_train.txt  
  inflating: minorR_dataset1/05_zyozu_train.txt  
  inflating: minorR_dataset1/09_samuke_test.txt  
  inflating: minorR_dataset1/

In [ ]:
# sentence BERTモデルの使用
!pip install -r minorR_dataset1/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 17.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 784.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.4/698.4 kB 47.0 MB/s eta 0:00:0

In [ ]:
#動作確認(サンプルプログラム)
from transformers import BertJapaneseTokenizer, BertModel
import torch
import scipy.spatial

class SentenceBertJapanese:
    def __init__(self, model_name_or_path, device=None):
        self.tokenizer = BertJapaneseTokenizer.from_pretrained(model_name_or_path)
        self.model = BertModel.from_pretrained(model_name_or_path)
        self.model.eval()

        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)
        self.model.to(device)

    def _mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    @torch.no_grad()
    def encode(self, sentences, batch_size=8):
        all_embeddings = []
        iterator = range(0, len(sentences), batch_size)
        for batch_idx in iterator:
            batch = sentences[batch_idx:batch_idx + batch_size]

            encoded_input = self.tokenizer.batch_encode_plus(batch, padding="longest",
                                           truncation=True, return_tensors="pt").to(self.device)
            model_output = self.model(**encoded_input)
            sentence_embeddings = self._mean_pooling(model_output, encoded_input["attention_mask"]).to('cpu')

            all_embeddings.extend(sentence_embeddings)

        return torch.stack(all_embeddings).numpy()
        # return torch.stack(all_embeddings)


### main routin

MODEL_NAME = "sonoisa/sentence-bert-base-ja-mean-tokens-v2"
model = SentenceBertJapanese(MODEL_NAME)

sentence_list = ['今日は雨がふっている。','今日は日差しがない。']

sentence_vectors = model.encode(sentence_list)

print('# 1=sentence, 2=first ten values of vector, 3=vector size')
for i in range(0,len(sentence_list)):
    print('1: '+sentence_list[i])
    print('2: '+str(sentence_vectors[i][0:10]))
    print('3: '+str(len(sentence_vectors[i])))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

# 1=sentence, 2=first ten values of vector, 3=vector size
1: 今日は雨がふっている。
2: [ 0.191 -1.114  0.698 -0.125  0.226 -0.36  -0.983 -1.572  0.582 -0.038]
3: 768
1: 今日は日差しがない。
2: [-0.409  0.249 -0.134  0.452 -0.448 -0.711 -0.871 -1.259 -0.005 -0.327]
3: 768


In [ ]:
# ファイルリスト
train_files=["minorR_dataset1/01_itimi_train.txt","minorR_dataset1/02_itimoku_train.txt","minorR_dataset1/03_ikken_train.txt","minorR_dataset1/04_itizu_train.txt","minorR_dataset1/05_zyozu_train.txt","minorR_dataset1/06_ninki_train.txt","minorR_dataset1/07_sizyo_train.txt","minorR_dataset1/08_sinzyu_train.txt","minorR_dataset1/09_samuke_train.txt","minorR_dataset1/10_mokka_train.txt"]
test_files=["minorR_dataset1/01_itimi_test.txt","minorR_dataset1/02_itimoku_test.txt","minorR_dataset1/03_ikken_test.txt","minorR_dataset1/04_itizu_test.txt","minorR_dataset1/05_zyozu_test.txt","minorR_dataset1/06_ninki_test.txt","minorR_dataset1/07_sizyo_test.txt","minorR_dataset1/08_sinzyu_test.txt","minorR_dataset1/09_samuke_test.txt","minorR_dataset1/10_mokka_test.txt"]

# 訓練データを読み込んで連結
train_dfs = [pd.read_table(f, header=None) for f in train_files]
datatrain = pd.concat(train_dfs, ignore_index=True)

# テストデータを読み込んで連結
test_dfs = [pd.read_table(f, header=None) for f in test_files]
datatest = pd.concat(test_dfs, ignore_index=True)

# カラム名を付与
column_names = ['id', 'target_word', 'reading', 'sentence']
datatrain.columns = column_names
datatest.columns = column_names

In [ ]:
# sentenceから*と/を削除
datatrain['sentence_del']=datatrain['sentence'].str.replace(r'[*\/]', '', regex=True)
datatest['sentence_del']=datatest['sentence'].str.replace(r'[*\/]', '', regex=True)

In [ ]:
# モデルの機械学習
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# 使用するカーネル関数
kernels = ['rbf','linear','poly','sigmoid']

# 結果を保存する辞書：{カーネル: {単語: 正解率}}
results = {}

# 文のベクトル化
from transformers import BertJapaneseTokenizer, BertModel
import torch
import scipy.spatial

class SentenceBertJapanese:
    def __init__(self, model_name_or_path, device=None):
        self.tokenizer = BertJapaneseTokenizer.from_pretrained(model_name_or_path)
        self.model = BertModel.from_pretrained(model_name_or_path)
        self.model.eval()

        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)
        self.model.to(device)

    def _mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    @torch.no_grad()
    def encode(self, sentences, batch_size=8):
        all_embeddings = []
        iterator = range(0, len(sentences), batch_size)
        for batch_idx in iterator:
            batch = sentences[batch_idx:batch_idx + batch_size]

            encoded_input = self.tokenizer.batch_encode_plus(batch, padding="longest",
                                           truncation=True, return_tensors="pt").to(self.device)
            model_output = self.model(**encoded_input)
            sentence_embeddings = self._mean_pooling(model_output, encoded_input["attention_mask"]).to('cpu')

            all_embeddings.extend(sentence_embeddings)

        return torch.stack(all_embeddings).numpy()
        # return torch.stack(all_embeddings)

### main routin

MODEL_NAME = "sonoisa/sentence-bert-base-ja-mean-tokens-v2"
model = SentenceBertJapanese(MODEL_NAME)

sentence_train_list = datatrain['sentence_del'].tolist()
sentence_test_list = datatest['sentence_del'].tolist()

sentence_train_vectors = model.encode(sentence_train_list)
sentence_test_vectors = model.encode(sentence_test_list)


# カーネルごとに処理
for kernel in kernels:
    print(f"\n カーネル: {kernel}")
    results[kernel] = {}

    for target_word in datatrain['target_word'].unique():
        # 対象単語に対応するデータを取り出す
        train_group = datatrain[datatrain['target_word'] == target_word]
        test_group  = datatest[datatest['target_word'] == target_word]
        # 異なる読み方が1種類以下なら分類不可能なのでスキップ
        if train_group['reading'].nunique() < 2:
            continue

        # Extract vectors corresponding to the current target word
        train_indices = train_group.index.tolist()
        test_indices = test_group.index.tolist()

        X_train = sentence_train_vectors[train_indices]
        y_train = train_group['reading']
        X_test = sentence_test_vectors[test_indices]
        y_test = test_group['reading']

        # SVMモデル（probability=True）
        model = SVC(kernel=kernel, probability=True)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # 正解率を保存
        acc = accuracy_score(y_test, y_pred)
        results[kernel][target_word] = acc

# 結果の表示
print("\n カーネルごとの対象単語別 正解率：")
for kernel in kernels:
    print(f"\n=== カーネル: {kernel} ===")
    for target_word, acc in results[kernel].items():
        print(f"{target_word:<5} -> 正解率: {acc:.4f}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]


🔍 カーネル: rbf

🔍 カーネル: linear

🔍 カーネル: poly

🔍 カーネル: sigmoid

📊 カーネルごとの対象単語別 正解率：

=== カーネル: rbf ===
一味    -> 正解率: 1.0000
一目    -> 正解率: 0.6667
一見    -> 正解率: 1.0000
一途    -> 正解率: 1.0000
上手    -> 正解率: 1.0000
人気    -> 正解率: 1.0000
市場    -> 正解率: 0.7500
心中    -> 正解率: 0.9000
寒気    -> 正解率: 1.0000
目下    -> 正解率: 0.6364

=== カーネル: linear ===
一味    -> 正解率: 1.0000
一目    -> 正解率: 0.6667
一見    -> 正解率: 0.9167
一途    -> 正解率: 0.9091
上手    -> 正解率: 0.9286
人気    -> 正解率: 0.9167
市場    -> 正解率: 0.8333
心中    -> 正解率: 0.8000
寒気    -> 正解率: 1.0000
目下    -> 正解率: 0.7273

=== カーネル: poly ===
一味    -> 正解率: 0.7500
一目    -> 正解率: 0.6667
一見    -> 正解率: 0.9167
一途    -> 正解率: 0.7273
上手    -> 正解率: 0.8571
人気    -> 正解率: 1.0000
市場    -> 正解率: 0.7500
心中    -> 正解率: 0.8000
寒気    -> 正解率: 0.9167
目下    -> 正解率: 0.7273

=== カーネル: sigmoid ===
一味    -> 正解率: 1.0000
一目    -> 正解率: 0.6667
一見    -> 正解率: 1.0000
一途    -> 正解率: 1.0000
上手    -> 正解率: 0.9286
人気    -> 正解率: 1.0000
市場    -> 正解率: 0.7500
心中    -> 正解率: 0.9000
寒気    -> 正解率: 1.0000
目下    -> 正解率: 0.

In [ ]:
# 文章から*を削除
datatrain['sentence']=datatrain['sentence'].str.replace(r'[*]', '', regex=True)
datatest['sentence']=datatest['sentence'].str.replace(r'[*]', '', regex=True)

# トークン化（スラッシュ区切り）と対象語の位置を取得
datatrain['sentence_tokens'] = datatrain['sentence'].apply(lambda s: s.split('/'))
datatest['sentence_tokens'] = datatest['sentence'].apply(lambda s: s.split('/'))

datatrain['target_index'] = datatrain.apply(
    lambda row: row['sentence_tokens'].index(row['target_word']) if row['target_word'] in row['sentence_tokens'] else -1,
    axis=1
)
datatest['target_index'] = datatest.apply(
    lambda row: row['sentence_tokens'].index(row['target_word']) if row['target_word'] in row['sentence_tokens'] else -1,
    axis=1
)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

def extract_feature_list_from_dataframe(df):
    feature_list = []

    for _, row in df.iterrows():
        tokens = row['sentence_tokens']
        index = row['target_index']
        if index == -1:
            continue  # 対象語が見つからない場合はスキップ

        # 各文に対して6個の素性を抽出（p1, p2, p3, n1, n2, n3）
        for n in range(1, 4):
            # 前方素性（p1, p2, p3）
            start = max(0, index - n)
            context = tokens[start:index]
            if len(context) < n:
                context = ['X'] * (n - len(context)) + context
            feature_list.append(f'p{n}:{ " ".join(context) }')

            # 後方素性（n1, n2, n3）
            end = index + 1 + n
            context = tokens[index + 1:end]
            if len(context) < n:
                context = context + ['X'] * (n - len(context))
            feature_list.append(f'n{n}:{ " ".join(context) }')

    return feature_list

def build_feature_dict_from_list(feature_list):
    feature_dict = {}
    for i, feature in enumerate(feature_list):
        feature_dict[feature] = i
    return feature_dict

    # 素性リストを datatrain + datatest から抽出
all_features = extract_feature_list_from_dataframe(pd.concat([datatrain, datatest], ignore_index=True))

# 重複を順序付きで除去
seen = set()
ordered_unique_features = []
for feat in all_features:
    if feat not in seen:
        seen.add(feat)
        ordered_unique_features.append(feat)

# 辞書を構築
feature_dict = build_feature_dict_from_list(ordered_unique_features)

def vectorize_data_by_feature_dict(df, feature_dict):
    vectors = []

    for _, row in df.iterrows():
        tokens = row['sentence_tokens']
        index = row['target_index']

        if index == -1:
            # 対象単語が見つからなければ全ゼロベクトル
            vectors.append(np.zeros(len(feature_dict), dtype=int))
            continue

        vec = np.zeros(len(feature_dict), dtype=int)

        for n in range(1, 4):
            # 前方素性 p1, p2, p3
            start = max(0, index - n)
            context = tokens[start:index]
            if len(context) < n:
                context = ['X'] * (n - len(context)) + context
            feat_str = f'p{n}:' + ' '.join(context)
            if feat_str in feature_dict:
                vec[feature_dict[feat_str]] += 1

            # 後方素性 n1, n2, n3
            end = index + 1 + n
            context = tokens[index + 1:end]
            if len(context) < n:
                context = context + ['X'] * (n - len(context))
            feat_str = f'n{n}:' + ' '.join(context)
            if feat_str in feature_dict:
                vec[feature_dict[feat_str]] += 1

        vectors.append(vec)

    return np.array(vectors)


def train_and_evaluate_per_word(datatrain, datatest, feature_dict):
    kernels = ['rbf', 'linear', 'poly', 'sigmoid']
    target_words = sorted(datatrain['target_word'].unique())

    # 結果保存用
    results = pd.DataFrame(index=target_words, columns=kernels)

    for word in target_words:
        print(f"\n=== 対象単語: {word} ===")

        # 各対象語に対するデータを抽出
        train_df = datatrain[datatrain['target_word'] == word]
        test_df = datatest[datatest['target_word'] == word]

        if train_df.empty or test_df.empty:
            print(f"  データ不足のためスキップ")
            continue

        # ベクトル化
        X_train = vectorize_data_by_feature_dict(train_df, feature_dict)
        y_train = train_df['reading']

        X_test = vectorize_data_by_feature_dict(test_df, feature_dict)
        y_test = test_df['reading']

        # カーネル別にSVM学習と評価
        for kernel in kernels:
            clf = SVC(kernel=kernel, probability=True)
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            acc = accuracy_score(y_test, y_pred)
            results.loc[word, kernel] = round(acc, 4)
            print(f"  ▶ カーネル {kernel}: 正解率 = {acc:.4f}")

    return results

# 実行
accuracy_table = train_and_evaluate_per_word(datatrain, datatest, feature_dict)


=== 対象単語: 一味 ===
  ▶ カーネル rbf: 正解率 = 0.9167
  ▶ カーネル linear: 正解率 = 1.0000
  ▶ カーネル poly: 正解率 = 0.7500
  ▶ カーネル sigmoid: 正解率 = 0.9167

=== 対象単語: 一目 ===
  ▶ カーネル rbf: 正解率 = 0.7500
  ▶ カーネル linear: 正解率 = 1.0000
  ▶ カーネル poly: 正解率 = 0.7500
  ▶ カーネル sigmoid: 正解率 = 1.0000

=== 対象単語: 一見 ===
  ▶ カーネル rbf: 正解率 = 0.9167
  ▶ カーネル linear: 正解率 = 1.0000
  ▶ カーネル poly: 正解率 = 0.7500
  ▶ カーネル sigmoid: 正解率 = 1.0000

=== 対象単語: 一途 ===
  ▶ カーネル rbf: 正解率 = 0.7273
  ▶ カーネル linear: 正解率 = 0.8182
  ▶ カーネル poly: 正解率 = 0.7273
  ▶ カーネル sigmoid: 正解率 = 0.8182

=== 対象単語: 上手 ===
  ▶ カーネル rbf: 正解率 = 0.5714
  ▶ カーネル linear: 正解率 = 0.6429
  ▶ カーネル poly: 正解率 = 0.5000
  ▶ カーネル sigmoid: 正解率 = 0.7143

=== 対象単語: 人気 ===
  ▶ カーネル rbf: 正解率 = 0.8333
  ▶ カーネル linear: 正解率 = 0.8333
  ▶ カーネル poly: 正解率 = 0.7500
  ▶ カーネル sigmoid: 正解率 = 0.7500

=== 対象単語: 寒気 ===
  ▶ カーネル rbf: 正解率 = 0.7500
  ▶ カーネル linear: 正解率 = 0.8333
  ▶ カーネル poly: 正解率 = 0.6667
  ▶ カーネル sigmoid: 正解率 = 0.7500

=== 対象単語: 市場 ===
  ▶ カーネル rbf: 正解率 = 0.6667
  ▶ カーネル linear: 

In [ ]:
def mixed_context_model(
    datatrain, datatest,
    feature_dict,                # 局所文脈用の素性辞書
    sentence_train_vectors,      # 大域文脈のベクトル（訓練）
    sentence_test_vectors,       # 大域文脈のベクトル（テスト）
    kernels=['rbf', 'linear', 'poly', 'sigmoid']
):
    results = {}
    mixed_predictions = {}

    for kernel in kernels:
        print(f"\nカーネル: {kernel}")
        results[kernel] = {}

        for target_word in datatrain['target_word'].unique():
            train_group = datatrain[datatrain['target_word'] == target_word]
            test_group = datatest[datatest['target_word'] == target_word]

            if train_group['reading'].nunique() < 2:
                continue  # 複数の読みがないならスキップ

            train_indices = train_group.index.tolist()
            test_indices = test_group.index.tolist()

            # ▼ 局所文脈ベクトル
            X_train_local = vectorize_data_by_feature_dict(train_group, feature_dict)
            X_test_local = vectorize_data_by_feature_dict(test_group, feature_dict)

            # ▼ 大域文脈ベクトル
            X_train_global = sentence_train_vectors[train_indices]
            X_test_global = sentence_test_vectors[test_indices]

            y_train = train_group['reading']
            y_test = test_group['reading']

            # 局所モデル
            clf_local = SVC(kernel=kernel, probability=True, random_state=42)
            clf_local.fit(X_train_local, y_train)
            prob_local = clf_local.predict_proba(X_test_local)
            pred_local = clf_local.classes_[np.argmax(prob_local, axis=1)]

            # 大域モデル
            clf_global = SVC(kernel=kernel, probability=True, random_state=42)
            clf_global.fit(X_train_global, y_train)
            prob_global = clf_global.predict_proba(X_test_global)
            pred_global = clf_global.classes_[np.argmax(prob_global, axis=1)]

            # ▼ 混合予測：どちらの確信度が高いかで選択
            max_prob_local = np.max(prob_local, axis=1)
            max_prob_global = np.max(prob_global, axis=1)

            mixed_pred = []
            for i in range(len(y_test)):
                if max_prob_local[i] >= max_prob_global[i]:
                    mixed_pred.append(pred_local[i])
                else:
                    mixed_pred.append(pred_global[i])

            # 保存・評価
            acc = accuracy_score(y_test, mixed_pred)
            results[kernel][target_word] = round(acc, 4)

            # 詳細予測保存（任意で）
            for idx, pred in zip(test_group.index, mixed_pred):
                mixed_predictions[(idx, kernel)] = pred

    return results, mixed_predictions

# 混合モデルを実行
mixed_results, mixed_predictions = mixed_context_model(
    datatrain,
    datatest,
    feature_dict,
    sentence_train_vectors,
    sentence_test_vectors
)

# 結果表示
accuracy_table = pd.DataFrame(mixed_results).T  # カーネル×対象語の正解率
print("\n=== 混合モデルの正解率（対象語×カーネル）===")
print(accuracy_table)


🧪 カーネル: rbf

🧪 カーネル: linear

🧪 カーネル: poly

🧪 カーネル: sigmoid

=== 混合モデルの正解率（対象語×カーネル）===
          一味      一目   一見      一途      上手   人気      市場   心中      寒気      目下
rbf      1.0  0.8333  1.0  1.0000  0.9286  1.0  0.8333  0.9  1.0000  0.8182
linear   1.0  0.8333  1.0  0.9091  1.0000  1.0  0.7500  0.9  1.0000  0.7273
poly     1.0  0.8333  1.0  1.0000  0.9286  1.0  0.8333  0.9  1.0000  0.6364
sigmoid  1.0  0.7500  1.0  1.0000  0.9286  1.0  0.8333  0.9  0.9167  0.7273
